In [634]:
#!g1.1
%pip install --upgrade gdown
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [93]:
#!g1.1
import gdown

url = 'https://drive.google.com/drive/folders/1qw05BnA1O-XDgJ50OgNGFSlTa9Kls00j?usp=sharing'
gdown.download_folder(url, quiet=True)

['/home/jupyter/work/resources/aapd_test.tsv',
 '/home/jupyter/work/resources/aapd_train.tsv',
 '/home/jupyter/work/resources/aapd_validation.tsv',
 '/home/jupyter/work/resources/label_test',
 '/home/jupyter/work/resources/label_train',
 '/home/jupyter/work/resources/label_val',
 '/home/jupyter/work/resources/text_test',
 '/home/jupyter/work/resources/text_train',
 '/home/jupyter/work/resources/text_val']

In [110]:
#!g1.1
!wget https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py -O wandb_writer.py

--2022-04-25 18:08:50--  https://gist.githubusercontent.com/ArseniyBolotin/7623835da1631b00fb150bcd5b0d909f/raw/wandb_writer.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2542 (2.5K) [text/plain]
Saving to: ‘wandb_writer.py’

wandb_writer.py     100%[===================>]   2.48K  --.-KB/s    in 0s      

2022-04-25 18:08:50 (49.6 MB/s) - ‘wandb_writer.py’ saved [2542/2542]



In [766]:
#!g1.1
# i need this one
def apply_to_dict_values(dict, f):
    for key, value in dict.items():
        dict[key] = f(value)

In [767]:
#!g1.1
BERT_TYPE = 'bert-base-uncased'

In [768]:
#!g1.1
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig

class AAPDDataset(Dataset):
    """AAPD dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True) # max_len=512 !DocBERT
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, AAPDDataset.target_to_tensor(self.data.iloc[idx, 0])

In [128]:
#!g1.1
train_dataset = AAPDDataset('./aapd_train.tsv')
val_dataset = AAPDDataset('./aapd_validation.tsv')
test_dataset = AAPDDataset('./aapd_test.tsv')

In [115]:
#!g1.1
BATCH_SIZE = 16 # !DocBERT
N_CLASSES = test_dataset[0][1].shape[0]
BATCH_SIZE, N_CLASSES

(16, 54)

In [676]:
#!g1.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [129]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [120]:
#!g1.1
!head aapd_train.tsv

000000000000000000001000000000000000000000000010000000	the relation between pearson 's correlation coefficient and salton 's cosine measure is revealed based on the different possible values of the division of the l1 norm and the l2 norm of a vector these different values yield a sheaf of increasingly straight lines which form together a cloud of points , being the investigated relation the theoretical results are tested against the author co citation relations among 24 informetricians for whom two matrices can be constructed , based on co citations the asymmetric occurrence matrix and the symmetric co citation matrix both examples completely confirm the theoretical results the results enable us to specify an algorithm which provides a threshold value for the cosine above which none of the corresponding pearson correlations would be negative using this threshold value can be expected to optimize the visualization of the vector space
11000000000000000000010000000000000000000000000000000

In [134]:
#!g1.1
!head HiAGM/data/rcv1.taxonomy

Root	CCAT	ECAT	GCAT	MCAT
CCAT	C11	C12	C13	C14	C15	C16	C17	C18	C21	C22	C23	C24	C31	C32	C33	C34	C41	C42
C15	C151	C152
C151	C1511
C17	C171	C172	C173	C174
C18	C181	C182	C183
C31	C311	C312	C313
C33	C331
C41	C411
ECAT	E11	E12	E13	E14	E21	E31	E41	E51	E61	E71


In [148]:
#!g1.1
!head train.tgt

ecat e21 e212
ccat c18 c181 c17 c171
mcat m12
mcat m14 m141
gcat ecat gpol e12 g15 g154
ccat c15 c151 c1511
ccat mcat c15 c152 m11 c17
gcat gpro gent gfas
ccat c15 c152
mcat m13 m131


In [714]:
#!g1.1
res = {}
with open('HiAGM/data/wos.taxnomy') as f:
    for line in f:
        tmp = line.strip().split('\t')
        for i in range(len(tmp)):
            if tmp[i] not in res:
                res[tmp[i]] = []
            if i != 0:
                res[tmp[0]].append(tmp[i])

In [715]:
#!g1.1
key2i = {}
for i, elem in enumerate(list(res.keys())):
    key2i[elem] = i
key2i

{'Root': 0,
 'CS': 1,
 'Medical': 2,
 'Civil': 3,
 'ECE': 4,
 'biochemistry': 5,
 'MAE': 6,
 'Psychology': 7,
 'Symbolic computation': 8,
 'Computer vision': 9,
 'Computer graphics': 10,
 'Operating systems': 11,
 'Machine learning': 12,
 'Data structures': 13,
 'network security': 14,
 'Image processing': 15,
 'Parallel computing': 16,
 'Distributed computing': 17,
 'Algorithm design': 18,
 'Computer programming': 19,
 'Relational databases': 20,
 'Software engineering': 21,
 'Bioinformatics': 22,
 'Cryptography': 23,
 'Structured Storage': 24,
 "Alzheimer's Disease": 25,
 "Parkinson's Disease": 26,
 'Sprains and Strains': 27,
 'Cancer': 28,
 'Sports Injuries': 29,
 'Senior Health': 30,
 'Multiple Sclerosis': 31,
 'Hepatitis C': 32,
 'Weight Loss': 33,
 'Low Testosterone': 34,
 'Fungal Infection': 35,
 'Diabetes': 36,
 'Parenting': 37,
 'Birth Control': 38,
 'Heart Disease': 39,
 'Allergies': 40,
 'Menopause': 41,
 'Emergency Contraception': 42,
 'Skin Care': 43,
 'Myelofibrosis': 44,

In [716]:
#!g1.1
N = len(key2i)

In [717]:
#!g1.1
res_new = {}
for k, v in res.items():
    res_new[key2i[k]] = [key2i[e] for e in v]
res_new

{0: [1, 2, 3, 4, 5, 6, 7],
 1: [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 2: [25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77],
 3: [78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88],
 4: [89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104],
 5: [105, 106, 107, 108, 109, 110, 111, 112, 113],
 6: [114, 115, 116, 117, 118, 119, 120, 121, 122],
 7: [123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 

In [642]:
#!g1.1
from tqdm import tqdm

with open('rcv1_v2_data/train.tgt', 'r') as f:
    with open('rcv1_v2_data/train.src', 'r') as g:
        with open('train_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 20833/20833 [00:01<00:00, 13580.97it/s]


In [643]:
#!g1.1
with open('rcv1_v2_data/valid.tgt', 'r') as f:
    with open('rcv1_v2_data/valid.src', 'r') as g:
        with open('valid_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    if e == 'm12ccat':
                        label[key2i['M12']] = '1'
                        label[key2i['CCAT']] = '1'
                        continue
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 2316/2316 [00:00<00:00, 225943.95it/s]


In [644]:
#!g1.1
with open('rcv1_v2_data/test.tgt', 'r') as f:
    with open('rcv1_v2_data/test.src', 'r') as g:
        with open('test_docbert.rcv', 'w') as fg:
            tgt_readlines = f.readlines()
            src_readlines = g.readlines()
            for i in tqdm(range(len(src_readlines))):
                label = ['0'] * N
                label[0] = '1'
                s = src_readlines[i]
                t = tgt_readlines[i].strip().split()
                for e in t:
                    label[key2i[e.upper()]] = '1'
                fg.write(''.join(label) + '\t' + s)

100%|██████████| 781265/781265 [00:03<00:00, 211549.76it/s]


In [645]:
#!g1.1
!head train_docbert.rcv

10101000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000110000000	recov recov recov recov excit excit bring mexic mexic mexic mexic mexic mexic mexic mexic mexic mexic market market market market market market market life emerg evident evident econom econom econom econom back back back track buzz tuesday tuesday tuesday tuesday tuesday stock stock stock clos record record high high high interest interest interest interest rate rate rate rate rate rate rate month low low low stag begin year ahead term term term term fundament fundament fundament matthew hickm lehm brother york point point point hist hist fall fall fall view etch mind invest cris decemb free peso peso peso stubborn week week quart gross domest domest produc report percent percent percent strong strong strong anal anal expect expect expect expect expect govern treasur bill cete cete second second fell level jan main pric index rally volum frenz million shar confound strength end long

In [139]:
#!g1.1
cnt = 0
with open('HiAGM/data/train.rcv') as f:
    for line in f:
        cnt += 1

In [140]:
#!g1.1
cnt

775220

In [141]:
#!g1.1
with open('HiAGM/data/valid.rcv') as f:
    for line in f:
        cnt += 1

In [142]:
#!g1.1
cnt

796730

In [143]:
#!g1.1
with open('HiAGM/data/test.rcv') as f:
    for line in f:
        cnt += 1

In [144]:
#!g1.1
cnt

797921

In [130]:
#!g1.1
for elem in train_dataloader:
    print(elem)
    break

[{'input_ids': tensor([[ 101, 2009, 2001,  ...,    0,    0,    0],
        [ 101, 1999, 2344,  ...,    0,    0,    0],
        [ 101, 2057, 4769,  ...,    0,    0,    0],
        ...,
        [ 101, 3728, 1010,  ...,    0,    0,    0],
        [ 101, 2005, 3435,  ...,    0,    0,    0],
        [ 101, 1996, 3291,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 

In [769]:
#!g1.1
class RCVDataset(Dataset):
    """RCV dataset."""

    def __init__(self, path):
        self.path = path
        self.data = pd.read_csv(self.path, sep='\t', header=None)
        self.tokenizer = BertTokenizer.from_pretrained(BERT_TYPE)

    def __len__(self):
        return self.data.shape[0]

    @staticmethod
    def target_to_tensor(target):
        return torch.tensor([float(label) for label in target])

    def __getitem__(self, idx):
        data = self.tokenizer(self.data.iloc[idx, 1], return_tensors="pt", max_length=512, padding="max_length", truncation=True) # max_len=512 !DocBERT
        apply_to_dict_values(data, lambda x: x.flatten())
        return data, RCVDataset.target_to_tensor(self.data.iloc[idx, 0])

In [771]:
#!g1.1
train_dataset = RCVDataset('./train_docbert.rcv')
val_dataset = RCVDataset('./valid_docbert.rcv')
test_dataset = RCVDataset('./test_docbert.rcv')

In [772]:
#!g1.1
BATCH_SIZE = 16 # !DocBERT
N_CLASSES = test_dataset[0][1].shape[0]
BATCH_SIZE, N_CLASSES

(16, 103)

In [773]:
#!g1.1
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [650]:
#!g1.1
for elem in train_dataloader:
    print(elem)
    break

[{'input_ids': tensor([[  101,  2358,  4817,  ...,     0,     0,     0],
        [  101,  2449,  2213,  ...,     0,     0,     0],
        [  101,  2358,  4817,  ...,     0,     0,     0],
        ...,
        [  101, 20631, 13089,  ...,     0,     0,     0],
        [  101,  2091,  2091,  ...,     0,     0,     0],
        [  101, 13988,  7068,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1

In [774]:
#!g1.1
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [765]:
#!g1.1
from models.structure_model.structure_encoder import StructureEncoder

In [727]:
#!g1.1
from torch import nn
class HierMax(nn.Module):
    def __init__(self, children):
        super(HierMax, self).__init__()
        self.children_nodes = children
    
    def forward(self, logits):
        logits = torch.transpose(logits, 0, 1)
        x = logits.clone()
        queue = [0]
        i = 0
        x[0] = 1.0
        while i < len(queue):
            node = queue[i]
            i += 1
            for child in self.children_nodes[node]:
                queue.append(child)
            if len(self.children_nodes[node]) > 0:
                z = x[node].clone()
                x[self.children_nodes[node]] = torch.softmax(logits[self.children_nodes[node]], -1) * z
        return torch.transpose(x, 0, 1)

In [326]:
#!g1.1
hiermax = HierMax(res_new)

In [221]:
#!g1.1
%pip install prettytable

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [222]:
#!g1.1
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        # if name.find("layer.0") == -1: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(bert)

Total Trainable Params: 109482240


109482240

In [729]:
#!g1.1
import torch.nn as nn

class MultiLabelBert(nn.Module):
    def __init__(self, bert_model, n_classes, res_new, freeze_bert_weights=False):
        super(MultiLabelBert, self).__init__()
        self.bert_model = bert_model
        if freeze_bert_weights: # == false !DocBERT
            for param in self.bert_model.parameters():
                param.requires_grad = False
        self.n_bert_features = bert_model.pooler.dense.out_features
        self.n_classes = n_classes
        self.dense = nn.Linear(self.n_bert_features, self.n_classes)
        self.activation = HierMax(res_new)

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        return self.activation(self.dense(bert_output.pooler_output))


In [731]:
#!g1.1
import torch.optim as optim

LEARNING_RATE = 2e-5 # !DocBERT

In [732]:
#!g1.1
%pip install wandb
from wandb_writer import WandbWriter

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [733]:
#!g1.1
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        optimizer,
        criterion,
        n_epochs,
        wandb_writer,
        wandb_iter_start = 0):

    wandb_iter = wandb_iter_start
    for epoch in range(1, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=False):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((logits > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            wandb_writer.set_step(wandb_iter)
            wandb_writer.add_scalar("Batch train loss", loss.item())
            wandb_iter += 1

        wandb_writer.add_scalar("Train loss", total_loss / len(train_dataloader))
        wandb_writer.add_scalar("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Train Hamming loss",(hamming_loss(all_targets, all_preds)))


        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cpu').tolist())
                all_preds.extend((logits > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            
            wandb_writer.add_scalar("Validation loss", total_loss / len(val_dataloader))
            wandb_writer.add_scalar("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            wandb_writer.add_scalar("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))
    print(wandb_iter)


In [734]:
#!g1.1
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 0.0 Gb


In [735]:
#!g1.1
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats('cuda')
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 0.0 Gb


In [736]:
#!g1.1
model = MultiLabelBert(bert, N_CLASSES, res_new).to(device)
optimizer = optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE, weight_decay=0.01)

In [737]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCELoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=0)

18800


In [740]:
#!g1.1
from sklearn.metrics import classification_report

def test_model(
        model,
        val_dataloader,
        test_dataloader,
        criterion,
        threshold):

    model.eval()

    with torch.no_grad():
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(test_dataloader, f"Val epoch#{0}", leave=False):
            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets[:, 1:].to(device)
            logits = model(batch)[:, 1:]
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > threshold).type(torch.DoubleTensor).to('cpu').tolist())
#             loss = criterion(logits, targets)
#             total_loss += loss.item()

#         print("Validation loss", total_loss / len(val_dataloader))
        print("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        print("Validation F1 (macro)",(f1_score(all_targets, all_preds, average='macro')))
        print("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))

In [742]:
#!g1.1
for i in range(50, 61):
    test_model(model, val_dataloader, test_dataloader, torch.nn.BCELoss(), i / 100)

Validation F1 (micro) 0.03153918299728875
Validation F1 (macro) 0.02825462206971487
Validation Hamming loss 0.8705064314323947
Validation F1 (micro) 0.750101219128166
Validation F1 (macro) 0.7132370338017757
Validation Hamming loss 0.008385051212209721
Validation F1 (micro) 0.7928846810794983
Validation F1 (macro) 0.7550034892292672
Validation Hamming loss 0.0064061489369249425
Validation F1 (micro) 0.7998436889409926
Validation F1 (macro) 0.7484264555672697
Validation Hamming loss 0.005798591220828739
Validation F1 (micro) 0.7717409682691766
Validation F1 (macro) 0.6976950303372463
Validation Hamming loss 0.006113313665067394


KeyboardInterrupt: 

In [743]:
#!g1.1
test_model(model, val_dataloader, test_dataloader, torch.nn.BCELoss(), 0.52)

Validation F1 (micro) 0.7918072053575793
Validation F1 (macro) 0.7521225179204034
Validation Hamming loss 0.0064287908393881555


In [703]:
#!g1.1
for i in range(1, 11):
    test_model(model, val_dataloader, test_dataloader, torch.nn.BCELoss(), i / 100)

Validation loss 0.06435467842966318
Validation Hamming loss 0.021781789638932497
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1274
           1       0.94      0.98      0.96       434
           2       0.43      0.94      0.58       158
           3       0.82      0.99      0.90       272
           4       0.96      0.98      0.97       410
           5       0.15      0.91      0.26        22
           6       0.04      1.00      0.07         1
           7       0.06      0.62      0.11         8
           8       0.07      1.00      0.14         5
           9       0.82      0.92      0.87       169
          10       0.00      1.00      0.00         0
          11       0.60      0.92      0.72        63
          12       0.69      0.93      0.79        85
          13       0.12      0.69      0.20        16
          14       0.12      0.83      0.21        12
          15       0.00      1.00      0.00         0


In [704]:
#!g1.1
config = BertConfig.from_pretrained(BERT_TYPE)
config.return_dict = True
bert = BertModel.from_pretrained(BERT_TYPE, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [705]:
#!g1.1
class MultiLabelBert(nn.Module):
    def __init__(self, bert_model, n_classes, freeze_bert_weights=False):
        super(MultiLabelBert, self).__init__()
        self.bert_model = bert_model
        if freeze_bert_weights: # == false !DocBERT
            for param in self.bert_model.parameters():
                param.requires_grad = False
        self.n_bert_features = bert_model.pooler.dense.out_features
        self.n_classes = n_classes
        self.dense = nn.Linear(self.n_bert_features, self.n_classes)

    def forward(self, inputs):
        bert_output = self.bert_model(**inputs)
        return self.dense(bert_output.pooler_output)

In [706]:
#!g1.1
import tqdm
from sklearn.metrics import f1_score, hamming_loss

def train_model(
        model,
        train_dataloader,
        val_dataloader,
        optimizer,
        criterion,
        n_epochs,
        wandb_writer,
        wandb_iter_start = 0):

    wandb_iter = wandb_iter_start
    for epoch in range(1, n_epochs+1):
        model.train()
        total_loss = 0.
        all_targets = []
        all_preds = []
        for batch, targets in tqdm.tqdm(train_dataloader, f"Train epoch#{epoch}", leave=False):

            apply_to_dict_values(batch, lambda x: x.to(device))
            targets = targets.to(device)
            logits = model(batch)
            all_targets.extend(targets.to('cpu').tolist())
            all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
            optimizer.zero_grad()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            wandb_writer.set_step(wandb_iter)
            wandb_writer.add_scalar("Batch train loss", loss.item())
            wandb_iter += 1

        wandb_writer.add_scalar("Train loss", total_loss / len(train_dataloader))
        wandb_writer.add_scalar("Train F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
        wandb_writer.add_scalar("Train Hamming loss",(hamming_loss(all_targets, all_preds)))


        model.eval()

        with torch.no_grad():
            total_loss = 0.
            all_targets = []
            all_preds = []
            for batch, targets in tqdm.tqdm(val_dataloader, f"Val epoch#{epoch}", leave=False):
                apply_to_dict_values(batch, lambda x: x.to(device))
                targets = targets.to(device)
                logits = model(batch)
                all_targets.extend(targets.to('cpu').tolist())
                all_preds.extend((torch.sigmoid(logits) > 0.5).type(torch.DoubleTensor).to('cpu').tolist())
                loss = criterion(logits, targets)
                total_loss += loss.item()
            
            wandb_writer.add_scalar("Validation loss", total_loss / len(val_dataloader))
            wandb_writer.add_scalar("Validation F1 (micro)",(f1_score(all_targets, all_preds, average='micro')))
            wandb_writer.add_scalar("Validation Hamming loss",(hamming_loss(all_targets, all_preds)))
    print(wandb_iter)


In [707]:
#!g1.1
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 13.74 Gb


In [708]:
#!g1.1
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats('cuda')
print("Cuda memory allocated: {:.4} Gb".format(torch.cuda.max_memory_allocated('cuda') / 1024 ** 3))

Cuda memory allocated: 1.664 Gb


In [709]:
#!g1.1
model = MultiLabelBert(bert, N_CLASSES).to(device)
optimizer = optim.AdamW(params=[p for p in model.parameters() if p.requires_grad], lr=LEARNING_RATE, weight_decay=0.01)

In [710]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=0)

Batch train loss,█▇▄▆▃▄▃▆▃▅▄▄▁▁▃▅▃▂▅▅▆▃▂▃▄▅▁▂▄▁▃▃▂▂▃▃▄▄▂▅
Train F1 (micro),▁▄▅▆▇▆▇▇██
Train Hamming loss,█▅▄▄▃▃▂▂▁▁
Train loss,█▄▃▂▂▁▂▁▁▁
Validation F1 (micro),▁▃▆▆█▆▇▅▇▇
Validation Hamming loss,█▆▃▄▁▃▃▄▃▂
Validation loss,█▅▃▂▃▃▁▂▂▂
steps_per_sec,▅▆▄▆▄█▇▆▇▅▅█▆▅▇▅▇▇▆▆▄▁▆▆▅▇▅▆▅▇▆▄▇▇▆▆▇▄▆▃
Batch train loss,96.15385
Train F1 (micro),0.47665
Train Hamming loss,0.024


7120


In [713]:
#!g1.1
test_model(model, val_dataloader, test_dataloader, torch.nn.BCEWithLogitsLoss(), 0.5)

Validation loss 0.016089470268343575
Validation Hamming loss 0.004279374471682164
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1274
           1       0.95      0.97      0.96       434
           2       0.98      0.85      0.91       158
           3       0.97      0.97      0.97       272
           4       0.97      0.98      0.97       410
           5       0.67      0.09      0.16        22
           6       1.00      0.00      0.00         1
           7       1.00      0.00      0.00         8
           8       1.00      0.00      0.00         5
           9       0.95      0.92      0.93       169
          10       1.00      1.00      1.00         0
          11       0.81      0.90      0.86        63
          12       0.95      0.87      0.91        85
          13       1.00      0.00      0.00        16
          14       1.00      0.00      0.00        12
          15       1.00      1.00      1.00         0

In [82]:
#!g1.1
torch.save(model.state_dict(), "DocBERT-base-10.pt")

In [83]:
#!g1.1
torch.save(optimizer.state_dict(), "DocBERT-base-10-opt.pt")

In [87]:
#!g1.1
model.load_state_dict(torch.load("DocBERT-base-10.pt"))

<All keys matched successfully>

In [88]:
#!g1.1
optimizer.load_state_dict(torch.load("DocBERT-base-10-opt.pt"))

In [89]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=33650)

Batch train loss,▆▂▁▃▃▄▄▇▆▅▅▆▅▇▅▃▅▄█
steps_per_sec,▁██████████▇████▇▇█
Batch train loss,0.02509
steps_per_sec,1.5931



67300


In [584]:
#!g1.1
a = torch.rand([8, 104])

In [585]:
#!g1.1
a

tensor([[1.2005e-02, 4.1000e-01, 8.2240e-02, 4.6471e-01, 4.3003e-01, 1.3229e-01,
         1.9726e-01, 1.2564e-01, 5.7225e-01, 8.5323e-01, 7.3326e-01, 9.8548e-01,
         2.1023e-01, 8.5376e-01, 1.6253e-01, 8.6058e-01, 3.8714e-01, 3.7135e-01,
         9.6583e-01, 6.6656e-01, 1.2696e-01, 8.2869e-01, 3.0563e-01, 8.5628e-01,
         6.0559e-01, 5.8053e-01, 7.8083e-02, 3.5552e-01, 2.7618e-01, 4.2456e-01,
         7.6638e-01, 8.8208e-01, 3.8489e-01, 2.5520e-01, 6.5931e-01, 1.2417e-01,
         8.9229e-01, 9.7431e-01, 8.9342e-02, 3.6683e-01, 3.3969e-01, 2.6000e-01,
         2.6992e-01, 2.4781e-01, 5.9398e-02, 3.7837e-01, 5.2555e-01, 7.6626e-02,
         1.4966e-01, 5.3640e-01, 1.8171e-01, 9.0757e-01, 3.6505e-01, 8.6780e-02,
         2.6935e-01, 4.3265e-01, 9.3028e-01, 9.8465e-01, 4.2523e-01, 8.3567e-01,
         1.7678e-01, 3.1137e-01, 5.8707e-01, 2.8761e-01, 4.2152e-01, 3.9154e-01,
         3.4811e-01, 4.3694e-01, 7.6345e-01, 6.3841e-01, 2.3005e-01, 4.0593e-01,
         3.2877e-01, 2.7596e

In [586]:
#!g1.1
hm = HierMax(res_new)
hm(a)

<ipython-input-95-76646eb2694f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(logits)
/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:864: UserWarning: The following variables cannot be serialized: model, optimizer
  warnings.warn(message)


tensor([[1.0000e+00, 1.2338e-01, 8.3670e-02, 1.2639e-01, 1.2422e-01, 9.7673e-03,
         1.2206e-02, 9.4817e-03, 2.2326e-02, 2.0985e-02, 1.8766e-02, 2.1328e-02,
         1.3279e-02, 1.8329e-02, 9.2070e-03, 2.1509e-02, 1.7086e-02, 1.4481e-02,
         2.0547e-02, 1.8519e-02, 1.4968e-02, 1.9981e-02, 1.1458e-02, 3.2514e-03,
         2.8155e-03, 3.9831e-04, 1.7069e-03, 2.1289e-03, 2.1753e-03, 2.6591e-03,
         2.0019e-03, 2.6249e-03, 1.5774e-03, 1.3910e-03, 1.7362e-03, 1.3854e-03,
         3.4060e-03, 3.4145e-03, 7.5881e-03, 9.1098e-03, 8.9988e-03, 6.8224e-03,
         6.8775e-03, 8.0993e-03, 6.8167e-03, 1.0135e-02, 9.0644e-03, 6.4046e-03,
         7.6416e-04, 1.0944e-03, 7.7643e-04, 1.4253e-03, 8.6311e-04, 5.9327e-04,
         6.1819e-04, 8.0108e-04, 1.3531e-03, 1.5488e-03, 9.1831e-04, 1.2784e-03,
         1.0102e-03, 1.1930e-03, 1.2122e-03, 1.2581e-02, 1.6858e-02, 1.4221e-02,
         1.4861e-02, 1.7145e-02, 2.2911e-02, 1.6523e-02, 1.3094e-02, 1.7328e-02,
         1.2219e-02, 1.0908e

In [91]:
#!g1.1
torch.save(model.state_dict(), "DocBERT-base-20.pt")
torch.save(optimizer.state_dict(), "DocBERT-base-20-opt.pt")

In [ ]:
!wget https://gist.githubusercontent.com/ArseniyBolotin/4187ea4be63cc9ef2944e07692ed8f89/raw/google_drive.py -O google_drive.py

In [ ]:
import google_drive
google_drive.auth()

In [ ]:
model_name = "BERT_AAPD_20.pt"
torch.save(model, model_name)

In [ ]:
google_drive.save_model(model_name)

In [ ]:
#!g1.1
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelDocBert"), wandb_iter_start=67300)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: c3n34ka (use `wandb login --relogin` to force relogin)
Train epoch#1:   1%|▏         | 45/3365 [00:25<30:20,  1.82it/s] 

In [ ]:
model_name = "BERT_AAPD_30.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)

In [ ]:
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 10, WandbWriter("MultiLabelBert"), wandb_iter_start=25249)

In [ ]:
model_name = "BERT_AAPD_40.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)

In [ ]:
train_model(model, train_dataloader, val_dataloader, optimizer, torch.nn.BCEWithLogitsLoss(), 20, WandbWriter("MultiLabelBert"), wandb_iter_start=25249 + 8420)

In [ ]:
model_name = "BERT_AAPD_60.pt"
torch.save(model, model_name)
google_drive.save_model(model_name)

In [ ]:
#!g1.1
bert

In [ ]:
#!g1.1
